<a href="https://colab.research.google.com/github/Osw1997/tsl/blob/main/examples/notebooks/replication_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm tsl -r
!git clone https://github.com/Osw1997/tsl.git

rm: cannot remove 'tsl': No such file or directory
Cloning into 'tsl'...
remote: Enumerating objects: 3531, done.
remote: Counting objects: 100% (705/705), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 3531 (delta 557), reused 518 (delta 467), pack-reused 2826
Receiving objects: 100% (3531/3531), 1.55 MiB | 5.58 MiB/s, done.
Resolving deltas: 100% (2407/2407), done.


In [ ]:
# import sys
# sys.path.insert(0, '/content')
# sys.path.append('/content')
'/content/tsl/tsl/datasets/raw_files_to_remove/testind3.ttl'

'/content/tsl/tsl/datasets/raw_files_to_remove/testind3.ttl'

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
# !pip install -q git+https://github.com/TorchSpatiotemporal/tsl.git


2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 39.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!rm tsl -r
!git clone https://github.com/Osw1997/tsl.git
!pip install ./tsl
# from tsl.datasets import dataobject_prototype

In [ ]:
# Agregarlo como dependencia en el archivo requirements.txt
!pip install kglab

In [ ]:
!pip install geopandas

In [1]:
# Lets import, if exists, numpy arrays from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# from tsl.tsl.datasets import dataobject_prototype
from tsl.datasets import dataobject_prototype

>>> CURRENT WORK DIRECTORY: [/content]


In [4]:
dataset = dataobject_prototype.CrimeMexicoCityTTL()

print(dataset)

Processing TTL and matrix distance
--> CURRENT WORK DIRECTORY: [/content]
CURRENT NAMESPASE: 
dist shape and dist: 
(16, 16)
[[    0.         12576.26436766 19056.14648476 23762.10105517
   7356.23026783  7698.93222475 14297.79377938 21280.66824408
  27277.26849452  7208.10364333 43324.94813224 33961.48101024
  31664.9104129  11835.15804703 30123.96036602 19019.65524512]
 [12576.26436766     0.          6479.88362851 17914.8988735
   6136.80847965 15418.20117571  7680.57638974 12193.78325477
  17779.26975308  7396.0078245  30964.78995188 22075.64507635
  21448.71997318  9590.07713414 17818.48961748 10884.87000356]
 [19056.14648476  6479.88362851     0.         17851.35973571
  12356.81180736 21289.20855584 10374.05804204 10749.87230057
  14796.32347946 13399.85110769 24683.12718205 16356.6275217
  17438.76111887 13811.59159376 11707.26256632 10793.82923822]
 [23762.10105517 17914.8988735  17851.35973571     0.
  21965.98447801 30315.45773348 25553.46516907 28414.37653505
   8080.123704

In [5]:
dataset.dataframe()

nodes,Azcapotzalco,Benito JuÃ¡rez,CoyoacÃ¡n,Cuajimalpa de Morelos,CuauhtÃ©moc,Gustavo A. Madero,Iztacalco,Iztapalapa,La Magdalena Contreras,Miguel Hidalgo,Milpa Alta,Tlalpan,TlÃ¡huac,Venustiano Carranza,Xochimilco,Ãlvaro ObregÃ³n
channels,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
date,,,,,,,,,,,,,,,,
2017-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-30,1,1,1,2,8,6,5,5,1,1,1,6,1,2,3,2
2022-11-06,2,3,6,2,5,4,3,5,1,2,1,2,2,2,3,3


In [4]:
dataset.dataframe() # old version

nodes,Azcapotzalco,Benito JuÃ¡rez,CoyoacÃ¡n,Cuajimalpa de Morelos,CuauhtÃ©moc,Gustavo A. Madero,Iztacalco,Iztapalapa,La Magdalena Contreras,Miguel Hidalgo,Milpa Alta,Tlalpan,TlÃ¡huac,Venustiano Carranza,Xochimilco,Ãlvaro ObregÃ³n
channels,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
date,,,,,,,,,,,,,,,,
1967-11-09,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1967-11-10,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1967-11-11,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1967-11-12,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1967-11-13,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-25,1,2,1,1,2,1,1,2,1,3,1,1,1,1,1,3
2022-11-26,1,1,1,1,2,1,1,1,1,3,1,1,1,1,1,3


In [6]:
dataset.dist.shape

(16, 16)

In [ ]:
# mask = (df.values != 0.).astype('uint8')
data

In [42]:
dft = dataset.dataframe().copy()
dft.sortlevel(1, ascending=False)

AttributeError: ignored

In [ ]:
print(f"Default similarity: {dataset.similarity_score}")
print(f"Available similarity options: {dataset.similarity_options}")
# print("==========================================")

sim = dataset.get_similarity("distance")  # or dataset.compute_similarity()
sim
# print("Similarity matrix W:")
# print_matrix(sim)

In [13]:
connectivity = dataset.get_connectivity(threshold=0.1,
                                        include_self=False,
                                        normalize_axis=1,
                                        layout="edge_index")
### Hay que comparar connectivity entre este dataset yu el nuesrto
connectivity

(array([[ 0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
          2,  2,  2,  2,  2,  2,  2,  3,  3,  4,  4,  4,  4,  4,  4,  4,
          4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,
          7,  7,  7,  7,  7,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,
          9, 10, 11, 11, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14,
         14, 14, 14, 15, 15, 15, 15, 15],
        [ 1,  4,  5,  6,  9, 13,  0,  2,  4,  6,  7,  9, 13, 15,  1,  4,
          6,  7,  8,  9, 13, 14, 15,  8, 15,  0,  1,  2,  5,  6,  7,  9,
         13,  0,  4,  6,  9, 13,  0,  1,  2,  4,  5,  7,  9, 13,  1,  2,
          4,  6, 12, 13, 14,  2,  3, 11, 15,  0,  1,  2,  4,  5,  6, 13,
         15, 14,  8, 14,  7, 14,  0,  1,  2,  4,  5,  6,  7,  9,  2,  7,
         10, 11, 12,  1,  2,  3,  8,  9]]),
 array([0.05612861, 0.1462486 , 0.31402537, 0.0366059 , 0.21023472,
        0.07343962, 0.08788005, 0.26355252, 0.17286341, 0.16046081,
        0.10410979, 0.20445333, 0.11973064, 0.15

In [41]:
# La matriz de distancisas debe de ser de tamanio " N X N" donde N es el tamaño
# de los nodos sensados. En este caso son las 16 alcadias. Posteriormente
# se trabajará a nivel de AGEB
import numpy as np
import pandas as pd
import kglab
import json
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point, MultiPolygon


f = open("/content/tsl/tsl/datasets/raw_files_to_remove/alcaldias_cdmx.json", encoding='utf8')
json_alcaldias = json.load(f)

### Mejor usamos directo la funcion de geopandas aunque eso del encoding no jala
geo_df = gpd.GeoDataFrame.from_features(json_alcaldias["features"], crs='EPSG:4326')
geo_df = geo_df.sort_values("nomgeo")
geo_df

,geometry,fid,nomgeo,cve_mun,cve_ent,cvegeo
5,"MULTIPOLYGON (((-99.18231 19.50748, -99.18229 ...",6,Azcapotzalco,002,09,09002
6,"MULTIPOLYGON (((-99.13679 19.40156, -99.13682 ...",7,Benito JuÃ¡rez,014,09,09014
12,"MULTIPOLYGON (((-99.13923 19.35687, -99.13772 ...",13,CoyoacÃ¡n,003,09,09003
7,"MULTIPOLYGON (((-99.25698 19.40043, -99.25694 ...",8,Cuajimalpa de Morelos,004,09,09004
3,"MULTIPOLYGON (((-99.12919 19.46247, -99.12884 ...",4,CuauhtÃ©moc,015,09,09015
9,"MULTIPOLYGON (((-99.11860 19.58446, -99.11691 ...",10,Gustavo A. Madero,005,09,09005
8,"MULTIPOLYGON (((-99.05584 19.42196, -99.05589 ...",9,Iztacalco,006,09,09006
4,"MULTIPOLYGON (((-99.05813 19.40072, -99.05814 ...",5,Iztapalapa,007,09,09007
11,"MULTIPOLYGON (((-99.20859 19.33610, -99.20867 ...",12,La Magdalena Contreras,008,09,09008
10,"MULTIPOLYGON (((-99.19058 19.46794, -99.19066 ...",11,Miguel Hidalgo,016,09,09016


In [82]:
geo_df.to_crs("EPSG:3857").iloc[0]["geometry"].distance(geo_df.to_crs("EPSG:3857").iloc[15]["geometry"])

6674.072998431988

In [92]:
# geo_df.to_crs("EPSG:3857").iloc[3]["geometry"].centroid
geo_df.to_crs("EPSG:3857").iloc[0]["geometry"].centroid.distance(geo_df.to_crs("EPSG:3857").iloc[3]["geometry"].centroid)

23762.10105517184

In [11]:
def distance_matrix_crimes(df):
        #cambiamos la proyección para distancias en metros
#         df = gpd.GeoDataFrame(
#             df, geometry = gpd.points_from_xy(df.lat, df.long),
#             crs="EPSG:4326"
#         )
        df = gpd.GeoDataFrame(df).to_crs('EPSG:3857')
#         df = df.set_crs('EPSG:4326').to_crs('EPSG:3857')
        dist = df.geometry.apply(lambda g: df.centroid.distance(g.centroid))

        # # Con esta funcion obtenemos la distancia de cada denuncia con respecto de cada centroide
        # return mat

        # Save to built directory (Update in the class definition)
        # path = os.path.join(self.root_dir, 'metr_la_dist.npy')
        path = "crime_cdmx_dist.npy"
        np.save(path, dist)

distance_matrix_crimes(geo_df)
path = 'crime_cdmx_dist.npy'
dist = np.load(path)
dist[dist == 0] = np.inf
dist

NameError: ignored

In [16]:
geo_df.crs

In [5]:
dataset.dist

AttributeError: ignored

In [5]:
dft = dataset.dataframe().copy()
dft.columns = dft.columns.get_level_values(0)
dft

nodes,uri,date,type,long,lat,atribute,crime,geometry,index_alcaldia,nombre_alcaldia
0,<http://localhost/100091>,2019-06-05,crime:obs,-99.072098,19.356560,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-99.0721 19.35656),4,Iztapalapa
1,<http://localhost/100124>,2019-02-10,crime:obs,-99.147743,19.475439,"[<http://localhost/VIOLENCIA_FAMILIAR>, 38.0, ...",<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (-99.14774 19.47544),5,Azcapotzalco
2,<http://localhost/10020>,2019-06-02,crime:obs,-99.108337,19.444241,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.10834 19.44424),2,Venustiano Carranza
3,<http://localhost/100449>,2019-05-18,crime:obs,-99.178902,19.444530,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.1789 19.44453),10,Miguel Hidalgo
4,<http://localhost/100505>,2019-06-07,crime:obs,-99.137573,19.491070,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.13757 19.49107),9,Gustavo A. Madero
...,...,...,...,...,...,...,...,...,...,...
13990,<http://localhost/99795>,2019-10-05,crime:obs,-99.058159,19.398790,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-99.05816 19.39879),4,Iztapalapa
13991,<http://localhost/99803>,2019-10-05,crime:obs,-99.157143,19.312111,[<http://localhost/ROBO_A_TRANSEUNTE_EN_VIA_PU...,<http://localhost/ROBO_A_TRANSEUNTE_EN_VÍA_PÚB...,POINT (-99.15714 19.31211),12,CoyoacÃ¡n
13992,<http://localhost/99881>,2019-10-06,crime:obs,-99.137993,19.279140,"[<http://localhost/VIOLACION>, 20.0, Femenino]",<http://localhost/VIOLACIÓN>,POINT (-99.13799 19.27914),15,Xochimilco
13993,<http://localhost/9992>,2019-06-03,crime:obs,-99.216240,19.446190,[<http://localhost/ROBO_A_TRANSEUNTE_SALIENDO_...,<http://localhost/ROBO_A_CUENTAHABIENTE_SALIEN...,POINT (-99.21624 19.44619),10,Miguel Hidalgo


In [144]:
dftc = dft.value_counts(["date", "nombre_alcaldia"]).unstack(fill_value=0)
# idx = pd.period_range(min(dftc.index), max(dftc.index))
# dftc.reindex(idx, fill_value=0)


# dftc.index.levels
# new_index = pd.MultiIndex.from_product(dftc.index.levels)
# dftc = dftc.reindex(new_index)
# dftc.resample('D')
dftc



# #Drop Index from MultiIndex
# df=df.droplevel(0, axis=0)
# print(df)


nombre_alcaldia,Azcapotzalco,Benito JuÃ¡rez,CoyoacÃ¡n,Cuajimalpa de Morelos,CuauhtÃ©moc,Gustavo A. Madero,Iztacalco,Iztapalapa,La Magdalena Contreras,Miguel Hidalgo,Milpa Alta,Tlalpan,TlÃ¡huac,Venustiano Carranza,Xochimilco,Ãlvaro ObregÃ³n
date,,,,,,,,,,,,,,,,
1967-11-09,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1969-12-31,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1971-01-01,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1979-01-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1983-03-07,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-25,0,0,1,0,0,1,0,2,0,0,1,0,0,0,0,3
2022-11-26,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0
2022-11-27,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2


In [147]:
dft = dataset.dataframe().copy()
dft.columns = dft.columns.get_level_values(0)
# # Hay que adaptar la info al formato necesario
# # 1. Conteo a nivel de delegacion por dia y por alcaldia (canal)
# dft.groupby(by=["date", "nombre_alcaldia"])["nombre_alcaldia"].count().unstack(fill_value=0)
# # 2. Obtener oldest, newest date del dataset
# idx = pd.period_range(min(df.date), max(df.date))
# results.reindex(idx, fill_value=0)
# # 3. Crear una tabla multiindex donde el indice horizontal son las fechas
# #    a nivel de dia, con las delegaciones como indice vertical y
# #    el tipo de delito como canal
# # 4. Dado que el indice horizontal no tendra datos todos los dias
# #    sera necesario rellenar tales dias vacios mediante pd.period_range
# # https://stackoverflow.com/questions/47231496/pandas-fill-missing-dates-in-time-series

# Se rellenan los dias faltantes con ceros
dftc = dft.value_counts(["date", "nombre_alcaldia"]).unstack(fill_value=0)
dftc = dftc.set_index(pd.DatetimeIndex(dftc.index))
dftc = dftc.resample('D').sum()
dftc

# FILTRAR DATOS A 2017

nombre_alcaldia,Azcapotzalco,Benito JuÃ¡rez,CoyoacÃ¡n,Cuajimalpa de Morelos,CuauhtÃ©moc,Gustavo A. Madero,Iztacalco,Iztapalapa,La Magdalena Contreras,Miguel Hidalgo,Milpa Alta,Tlalpan,TlÃ¡huac,Venustiano Carranza,Xochimilco,Ãlvaro ObregÃ³n
date,,,,,,,,,,,,,,,,
1967-11-09,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1967-11-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1967-11-11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1967-11-12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1967-11-13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-25,0,0,1,0,0,1,0,2,0,0,1,0,0,0,0,3
2022-11-26,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0
2022-11-27,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2


In [ ]:
# Organizar la funcion de compute_similarity para considerar delegacion

In [ ]:
# Añadir matrix "covariates"

In [ ]:
dataset.get_connectivity(threshold=0.1, include_self=False, normalize_axis=1, layout='edge_index')

AttributeError: ignored

In [43]:
import os
print(f"Current path: {os.getcwd()}")

import numpy as np
import pandas as pd
import kglab
import json
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point, MultiPolygon
from shapely import wkt

def contained(geo_df, points_series):
    # Se obtienen los indices/nombre de las alcaldias a la que pertenecen cada punto
    # el conteo puede llevarse a cabo despues con un built-in method "group-by"
    list_belonging = list(map(geo_df['geometry'].contains, points_series))
    index_geom = [-1 if x.sum() == 0 else geo_df[x].index[0] for x in list_belonging]
    name_geo = ['NA' if x.sum() == 0 else geo_df[x]['nomgeo'].item() for x in list_belonging]
    return index_geom, name_geo

#### La matriz de sitancias entre denuncias y denuncias denuncias las podemos calcular asi
def distance_matrix_crimes(df):
    df = gpd.GeoDataFrame(df).to_crs('EPSG:3857')
    dist = df.geometry.apply(lambda g: df.centroid.distance(g.centroid))

    path = "crime_cdmx_dist.npy"
    np.save(path, dist)

def load_ttl_sparql():
    NAMESPACES = {
        "wtm":  "http://purl.org/heals/food/",
        "ind":  "http://purl.org/heals/ingredient/",
        "recipe":  "https://www.food.com/recipe/",

        "crime": "http://localhost/ontology2#",
        "cube": "http://purl.org/linked-data/cube#",
        "geo": "http://www.w3.org/2003/01/geo/wgs84_pos#",
        "ontology": "http://dbpedia.org/ontology/",
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "xsd": "http://www.w3.org/2001/XMLSchema#"
    }

    kg = kglab.KnowledgeGraph(namespaces = NAMESPACES)
    print(f"--> CURRENT WORK DIRECTORY: [{os.getcwd()}]")
    _ = kg.load_rdf("/content/tsl/tsl/datasets/raw_files_to_remove/testind3.ttl")

    print("CURRENT NAMESPASE: ")
    kg.describe_ns()

    # Buscamos tener id, delito, atributos, coordenadas y fecha en el DF
    # A clasificar: crime:tieneCategoria
    # Atributo: crime:contiene, crime:edad, crime:genero
    sparql_contiene = """
        SELECT distinct *
        WHERE {
            ?uri a crime:obs .
            ?uri a ?type .
            ?uri crime:tieneFecha ?date .
            ?uri geo1:lat ?lat .
            ?uri geo1:long ?long .

            ?uri crime:contiene ?atribute .
        }
    """
    sparql_edad = """
        SELECT distinct *
        WHERE {
            ?uri a crime:obs .
            ?uri a ?type .
            ?uri crime:tieneFecha ?date .
            ?uri geo1:lat ?lat .
            ?uri geo1:long ?long .

            ?uri crime:edad ?atribute .
        }
    """
    sparql_genero = """
        SELECT distinct *
        WHERE {
            ?uri a crime:obs .
            ?uri a ?type .
            ?uri crime:tieneFecha ?date .
            ?uri geo1:lat ?lat .
            ?uri geo1:long ?long .

            ?uri crime:genero ?atribute
        }
    """

    df_contiene = kg.query_as_df(sparql=sparql_contiene)
    df_edad = kg.query_as_df(sparql=sparql_edad)
    df_genero = kg.query_as_df(sparql=sparql_genero)

    df_atributes = pd.concat([df_contiene, df_edad, df_genero], ignore_index=True)
    df_atributes = df_atributes.groupby(['uri', 'date', 'type', 'long', 'lat'])['atribute'].apply(list).reset_index(name='atribute')
    # df_atributes

    # Luego obtengamos el uri, delito a clasificar
    sparql_crimes = """
        SELECT distinct *
        WHERE {
            ?uri crime:tieneCategoria ?crime .
        }
    """
    df_crimes = kg.query_as_df(sparql=sparql_crimes)
    # Join by uri
    df = pd.merge(df_atributes, df_crimes, how="inner", on="uri")
    # Se le crea la geometria "Point" a cada crimen segun su longitud/latitud
    df["geometry"] = df[["long", "lat"]].T.apply(Point)

    return df

def load_raw(geo_detail):
    path_ttl_sparql = "/content/drive/MyDrive/tsl_datasets/processed_ttl_sparql.csv"
    if os.path.isfile(path_ttl_sparql):
        df = pd.read_csv(path_ttl_sparql)
        # Convert "geometry" column <str> to <geopandas>
        df["geometry"] = gpd.GeoSeries.from_wkt(df["geometry"])
    else:
        df = load_ttl_sparql()
        df.to_csv(path_ttl_sparql, index=False)

    # Carga geometrias de alcaldias
    if geo_detail == 'alcaldia':
        path_geojson = "/content/tsl/tsl/datasets/raw_files_to_remove/alcaldias_cdmx.json"
    elif geo_detail == 'ageb':
        path_geojson = "/content/tsl/tsl/datasets/raw_files_to_remove/agebs.json"
    f = open(path_geojson, encoding='utf8')


    json_alcaldias = json.load(f)
    # geo_df = gpd.GeoDataFrame.from_features(json_alcaldias["features"])
    geo_df = gpd.GeoDataFrame.from_features(json_alcaldias["features"], crs='EPSG:4326')

    # TO REMOVE ONCE JSON HAS TE APPROPRIATE FORMAT (agebs)
    # ========================================================================================
    # ========================================================================================
    if geo_detail == 'ageb':
        geo_df["nomgeo"] = geo_df["id"] + "_" + geo_df["cve_ageb"]
        del geo_df["id"], geo_df["cve_ageb"]
    # ========================================================================================
    # ========================================================================================
    geo_df = geo_df.sort_values("nomgeo")

    # # Encuentra su respectiva alcaldia y las respectivas coordenadas a evaluar
    # if aggregation_level == "alcaldia":
    # Se invoca la funcion y se le agrega la columnas al dataframe
    index_geom, name_geo = contained(geo_df, df["geometry"])
    df["index_alcaldia"] = index_geom
    df["nombre_alcaldia"] = name_geo
    # Let's remove empty lat,long/Invalid rows
    valid_rows = (df["index_alcaldia"] != -1)
    df = df[valid_rows]

    # Carga la matriz de distancias desde el archivo .npy
#         path = os.path.join(self.root_dir, 'crime_cdmx_dist.npy')
    path = 'crime_cdmx_dist.npy'
    dist = np.load(path, allow_pickle=True)

    # Lets convert the dataframe into another dataframe but using right format
    df_raw = df.copy() # TO REMOVE: Only for testing purposes
    df = df.value_counts(["date", "nombre_alcaldia"]).unstack(fill_value=0)
    df = df.set_index(pd.DatetimeIndex(df.index))
    df = df.resample('D').sum()

    # Agregacion semanal
    df['date'] = df.index - pd.to_timedelta(7, unit='d')
    df = df.groupby([pd.Grouper(key='date', freq='W')]).sum()

    # Con esta funcion obtenemos la distancia de cada denuncia con respecto a cada denuncia
    # Debido a que no todos las geometrias pueden ser usadas, existira una
    # incogruencia entre la matriz de distancias respecto a las columnas
    # del dataframe ocasionando un error al agregar la matriz de covarianza
    # por lo que se filtran el dataframe de geometrias con las geometrias que
    # se utilizaron.
    used_geometries = list(df.columns.get_level_values(0))
    geo_df = geo_df[geo_df["nomgeo"].isin(used_geometries)]
    # df[df['A'].isin([3, 6])
    distance_matrix_crimes(geo_df)

    # Filter rows < 2017
    df = df[df.index >= '2017-01-01']

    return df, dist
    # return df, dist, df_raw

Current path: /content


In [45]:
path = 'crime_cdmx_dist.npy'
dist = np.load(path)
dist.shape

(2144, 2144)

In [46]:
dft, distt = load_raw(geo_detail='ageb')

In [47]:
dft

nombre_alcaldia,agebs.0_0543,agebs.1000_1636,agebs.1001_465A,agebs.1002_1640,agebs.1003_1655,agebs.1004_166A,agebs.1005_1674,agebs.1006_1689,agebs.1007_1693,agebs.1008_1706,...,agebs.990_586A,agebs.992_5836,agebs.993_5840,agebs.995_1301,agebs.996_1091,agebs.997_1246,agebs.998_1369,agebs.999_2691,agebs.99_0666,agebs.9_0670
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-11-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-11-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
distt.shape

(2144, 2144)

In [26]:
used_geometries = list(dft.columns.get_level_values(0))


In [42]:
# Carga geometrias de alcaldias
f = open("/content/tsl/tsl/datasets/raw_files_to_remove/agebs.json", encoding='utf8')
json_alcaldias = json.load(f)
json_alcaldias

geo_df = gpd.GeoDataFrame.from_features(json_alcaldias["features"], crs='EPSG:4326')
geo_df["nomgeo"] = geo_df["id"] + "_" + geo_df["cve_ageb"]
del geo_df["id"], geo_df["cve_ageb"]
# Filtra por geometrias usadas
geo_df = geo_df[geo_df["nomgeo"].isin(used_geometries)]
# geo_df.to_csv("agebs_nomgeo.json", index=False)
geo_df = geo_df.sort_values("nomgeo")
distance_matrix_crimes(geo_df)
# geo_df["nomgeo"].unique


In [14]:
distt.shape

(2453, 2453)

In [4]:
distance_matrix_crimes(geo_df)

In [6]:
# df = gpd.GeoDataFrame(df).to_crs('EPSG:3857')
# dist = df.geometry.apply(lambda g: df.centroid.distance(g.centroid))
gpd.GeoDataFrame(geo_df).to_crs('EPSG:3857')

,geometry,fid,cve_mun,cve_ent,cvegeo,nomgeo
0,"POLYGON ((inf inf, inf inf, inf inf, inf inf, ...",0,014,09,0901400010543,agebs.0_0543
1000,"POLYGON ((inf inf, inf inf, inf inf, inf inf, ...",1000,007,09,0900700011636,agebs.1000_1636
1001,"POLYGON ((inf inf, inf inf, inf inf, inf inf, ...",1001,007,09,090070001465A,agebs.1001_465A
1002,"POLYGON ((inf inf, inf inf, inf inf, inf inf, ...",1002,007,09,0900700011640,agebs.1002_1640
1003,"POLYGON ((inf inf, inf inf, inf inf, inf inf, ...",1003,007,09,0900700011655,agebs.1003_1655
...,...,...,...,...,...,...
997,"POLYGON ((inf inf, inf inf, inf inf, inf inf, ...",997,007,09,0900700011246,agebs.997_1246
998,"POLYGON ((inf inf, inf inf, inf inf, inf inf, ...",998,007,09,0900700011369,agebs.998_1369
999,"POLYGON ((inf inf, inf inf, inf inf, inf inf, ...",999,007,09,0900700012691,agebs.999_2691
99,"POLYGON ((inf inf, inf inf, inf inf, inf inf, ...",99,014,09,0901400010666,agebs.99_0666


In [5]:
np.load("/content/crime_cdmx_dist.npy")

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [33]:
dft, geo_dft = load_raw()
# dft

In [46]:
dft, dist = load_raw()
dft

nombre_alcaldia,Azcapotzalco,Benito JuÃ¡rez,CoyoacÃ¡n,Cuajimalpa de Morelos,CuauhtÃ©moc,Gustavo A. Madero,Iztacalco,Iztapalapa,La Magdalena Contreras,Miguel Hidalgo,Milpa Alta,Tlalpan,TlÃ¡huac,Venustiano Carranza,Xochimilco,Ãlvaro ObregÃ³n
date,,,,,,,,,,,,,,,,
2017-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-30,1,1,1,0,8,6,0,5,0,1,1,6,1,2,3,2
2022-11-06,2,3,6,0,5,4,3,5,0,2,0,2,2,2,0,3
2022-11-13,2,4,7,0,10,3,2,6,2,2,0,1,2,1,0,4


In [44]:
dft['date'] = dft.index - pd.to_timedelta(7, unit='d')
dft.groupby([pd.Grouper(key='date', freq='W')]).sum()

# dft.to_csv("t.csv")

nombre_alcaldia,Azcapotzalco,Benito JuÃ¡rez,CoyoacÃ¡n,Cuajimalpa de Morelos,CuauhtÃ©moc,Gustavo A. Madero,Iztacalco,Iztapalapa,La Magdalena Contreras,Miguel Hidalgo,Milpa Alta,Tlalpan,TlÃ¡huac,Venustiano Carranza,Xochimilco,Ãlvaro ObregÃ³n
date,,,,,,,,,,,,,,,,
2016-12-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-30,1,1,1,0,8,6,0,5,0,1,1,6,1,2,3,2
2022-11-06,2,3,6,0,5,4,3,5,0,2,0,2,2,2,0,3
2022-11-13,2,4,7,0,10,3,2,6,2,2,0,1,2,1,0,4


In [ ]:
path = os.path.join('', 'crime_cdmx_dist.npy')
dist = np.load(path)
# (dist[-210] == np.Inf).sum()


array([[    0.        , 16364.1972812 , 11106.62506666, ...,
        11713.37346392, 19218.75442714,  5937.86741752],
       [16364.1972812 ,     0.        ,  5727.62988752, ...,
        23189.59567573,  8370.92540425, 21698.6718701 ],
       [11106.62506666,  5727.62988752,     0.        , ...,
        19758.22562267, 12013.57883742, 16794.48393614],
       ...,
       [11713.37346392, 23189.59567573, 19758.22562267, ...,
            0.        , 21549.76892587,  8063.84562384],
       [19218.75442714,  8370.92540425, 12013.57883742, ...,
        21549.76892587,     0.        , 23074.87270322],
       [ 5937.86741752, 21698.6718701 , 16794.48393614, ...,
         8063.84562384, 23074.87270322,     0.        ]])

In [ ]:
dist[0]

array([    0.        , 16364.1972812 , 11106.62506666, ...,
       11713.37346392, 19218.75442714,  5937.86741752])

In [ ]:
def distance_matrix_crimes(df):
    #cambiamos la proyección para distancias en metros
    df = gpd.GeoDataFrame(
        df, geometry = gpd.points_from_xy(df.long, df.lat),
        crs="EPSG:4326"
    )
    df = df.set_crs('EPSG:4326').to_crs('EPSG:3857')
    dist = df.geometry.apply(lambda g: df.distance(g))

dft10 = dft[:].copy()

gdft = gpd.GeoDataFrame(dft10, crs='EPSG:4326').to_crs('EPSG:3857')
gdft.geometry.apply(lambda g: gdft.distance(g))
# gpd.GeoDataFrame(dft10, geometry = gpd.points_from_xy(dft[:10].long, dft[:10].lat), crs="EPSG:4326")
#

,0,1,2,3,4,5,6,7,9,10,...,13984,13985,13986,13987,13989,13990,13991,13992,13993,13994
0,0.000000,16364.197281,11106.625067,15784.117603,17470.068525,16621.762500,17974.102145,18662.245619,18201.036320,5825.580419,...,12934.256165,8886.113045,2563.158266,12641.728682,9595.751644,5219.340306,10821.973354,11713.373464,19218.754427,5937.867418
1,16364.197281,0.000000,5727.629888,5034.857950,2165.167672,6321.092568,7985.720802,6530.113484,22485.713515,21335.339564,...,29211.338055,8899.602277,18878.572002,6923.647153,8839.487395,13465.252776,19303.920375,23189.595676,8370.925404,21698.671870
2,11106.625067,5727.629888,0.000000,7854.777878,6415.581414,9255.246957,11115.099856,10580.706571,21428.230166,16518.922849,...,23727.756173,6025.659933,13668.005401,3225.364734,6538.937497,7744.928830,16511.197740,19758.225623,12013.578837,16794.483936
3,15784.117603,5034.857950,7854.777878,0.000000,7166.748699,1447.142116,3312.172212,2955.380645,17894.092243,19758.460841,...,28630.670146,6980.511541,18050.620170,10451.535633,6440.890010,14484.533771,15812.815494,20039.162515,4161.286745,20231.824271
4,17470.068525,2165.167672,6415.581414,7166.748699,0.000000,8400.143013,9981.680995,8348.760465,24624.247044,22699.509906,...,30142.821916,10660.673212,20022.072547,6546.379625,10706.189698,14029.300583,21233.196938,25009.830569,10235.878815,23021.921332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13990,5219.340306,13465.252776,7744.928830,14484.533771,14029.300583,15657.412699,17336.937108,17430.763142,21879.165785,10985.908452,...,16362.978643,8748.380694,7442.767107,8204.034317,9619.315717,0.000000,15033.330710,16680.215898,18465.408558,11020.986068
13991,10821.973354,19303.920375,16511.197740,15812.815494,21233.196938,15707.209195,15983.408518,17698.797176,7577.780105,8932.871192,...,18364.926258,10705.017634,10677.129381,19414.349599,10536.811104,15033.330710,0.000000,4434.634714,17135.443487,9560.817828
13992,11713.373464,23189.595676,19758.225623,20039.162515,25009.830569,20028.771790,20384.422834,22054.232558,9828.458181,7558.403585,...,15646.119961,14350.230003,10568.232937,22395.364589,14350.422910,16680.215898,4434.634714,0.000000,21549.768926,8063.845624
13993,19218.754427,8370.925404,12013.578837,4161.286745,10235.878815,2824.313502,1266.720437,1898.453450,17399.266221,22545.754365,...,31823.809719,10349.069468,21305.171039,14516.797891,9624.008028,18465.408558,17135.443487,21549.768926,0.000000,23074.872703


In [ ]:
dft[:10]

,uri,date,type,long,lat,atribute,crime,geometry,index_alcaldia,nombre_alcaldia
0,<http://localhost/100091>,2019-06-05,crime:obs,-99.07210,19.35656,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-99.0721 19.35656),4,Iztapalapa
1,<http://localhost/100124>,2019-02-10,crime:obs,-99.14774,19.47544,"[<http://localhost/VIOLENCIA_FAMILIAR>, 38.0, ...",<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (-99.14774 19.47544),5,Azcapotzalco
2,<http://localhost/10020>,2019-06-02,crime:obs,-99.10834,19.44424,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.10834 19.44424),2,Venustiano Carranza
3,<http://localhost/100449>,2019-05-18,crime:obs,-99.17890,19.44453,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.1789 19.44453),10,Miguel Hidalgo
4,<http://localhost/100505>,2019-06-07,crime:obs,-99.13757,19.49107,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.13757 19.49107),9,Gustavo A. Madero
5,<http://localhost/100557>,2019-06-07,crime:obs,-99.19142,19.44123,[<http://localhost/ROBO_A_TRANSEUNTE_A_BORDO_D...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_T...,POINT (-99.19142 19.44123),10,Miguel Hidalgo
6,<http://localhost/100644>,2019-05-28,crime:obs,-99.20802,19.43877,[<http://localhost/ROBO_A_TRANSEUNTE_SALIENDO_...,<http://localhost/ROBO_A_CUENTAHABIENTE_SALIEN...,POINT (-99.20802 19.43877),10,Miguel Hidalgo
7,<http://localhost/100665>,2019-03-08,crime:obs,-99.20259,19.45583,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.20259 19.45583),10,Miguel Hidalgo
9,<http://localhost/100685>,2019-06-08,crime:obs,-99.22376,19.29891,[<http://localhost/HOMICIDIO_POR_ARMA_DE_FUEGO...,<http://localhost/HOMICIDIO_DOLOSO>,POINT (-99.22376 19.29891),0,Tlalpan
10,<http://localhost/100728>,2019-06-06,crime:obs,-99.07705,19.30740,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.07705 19.3074),4,Iztapalapa


In [ ]:
dft[:10]

,uri,date,type,long,lat,atribute,crime,geometry
0,<http://localhost/100091>,2019-06-05,crime:obs,-99.07210,19.35656,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-99.0721 19.35656)
1,<http://localhost/100124>,2019-02-10,crime:obs,-99.14774,19.47544,"[<http://localhost/VIOLENCIA_FAMILIAR>, 38.0, ...",<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (-99.14774 19.47544)
2,<http://localhost/10020>,2019-06-02,crime:obs,-99.10834,19.44424,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.10834 19.44424)
3,<http://localhost/100449>,2019-05-18,crime:obs,-99.17890,19.44453,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.1789 19.44453)
4,<http://localhost/100505>,2019-06-07,crime:obs,-99.13757,19.49107,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.13757 19.49107)
5,<http://localhost/100557>,2019-06-07,crime:obs,-99.19142,19.44123,[<http://localhost/ROBO_A_TRANSEUNTE_A_BORDO_D...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_T...,POINT (-99.19142 19.44123)
6,<http://localhost/100644>,2019-05-28,crime:obs,-99.20802,19.43877,[<http://localhost/ROBO_A_TRANSEUNTE_SALIENDO_...,<http://localhost/ROBO_A_CUENTAHABIENTE_SALIEN...,POINT (-99.20802 19.43877)
7,<http://localhost/100665>,2019-03-08,crime:obs,-99.20259,19.45583,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.20259 19.45583)
9,<http://localhost/100685>,2019-06-08,crime:obs,-99.22376,19.29891,[<http://localhost/HOMICIDIO_POR_ARMA_DE_FUEGO...,<http://localhost/HOMICIDIO_DOLOSO>,POINT (-99.22376 19.29891)
10,<http://localhost/100728>,2019-06-06,crime:obs,-99.07705,19.30740,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.07705 19.3074)


In [ ]:
def contained(geo_df, points_series):
        # Se obtienen los indices/nombre de las alcaldias a la que pertenecen cada punto
        # el conteo puede llevarse a cabo despues con un built-in method "group-by"
        list_belonging = list(map(geo_df['geometry'].contains, points_series))
        # return list_belonging
        index_geom = [-1 if x.sum() == 0 else geo_df[x].index[0] for x in list_belonging]
        name_geo = ['NA' if x.sum() == 0 else geo_df[x]['nomgeo'].item() for x in list_belonging]
        return index_geom, name_geo

# ig, ng = contained(gd, dft["geometry"])
lc = contained(gd, dft["geometry"])

In [ ]:
lc[0]

[4,
 5,
 2,
 10,
 9,
 10,
 10,
 10,
 0,
 4,
 9,
 8,
 5,
 10,
 4,
 3,
 9,
 9,
 9,
 9,
 8,
 9,
 3,
 10,
 9,
 6,
 6,
 12,
 0,
 1,
 4,
 9,
 3,
 9,
 4,
 14,
 4,
 9,
 12,
 14,
 9,
 10,
 2,
 3,
 15,
 5,
 0,
 10,
 6,
 0,
 4,
 12,
 4,
 2,
 4,
 5,
 4,
 4,
 2,
 2,
 12,
 8,
 2,
 14,
 11,
 9,
 6,
 10,
 4,
 6,
 2,
 12,
 4,
 8,
 12,
 4,
 9,
 13,
 3,
 6,
 8,
 9,
 9,
 15,
 15,
 8,
 3,
 4,
 6,
 8,
 4,
 8,
 9,
 9,
 12,
 15,
 1,
 9,
 4,
 0,
 3,
 12,
 4,
 6,
 10,
 14,
 9,
 4,
 9,
 10,
 14,
 0,
 4,
 9,
 12,
 14,
 6,
 8,
 4,
 3,
 3,
 4,
 9,
 4,
 6,
 3,
 4,
 3,
 2,
 3,
 3,
 10,
 10,
 3,
 9,
 8,
 10,
 4,
 10,
 10,
 9,
 8,
 7,
 5,
 9,
 4,
 8,
 4,
 4,
 4,
 5,
 14,
 0,
 8,
 6,
 12,
 14,
 3,
 3,
 5,
 3,
 12,
 14,
 8,
 12,
 4,
 3,
 4,
 3,
 3,
 4,
 4,
 12,
 14,
 9,
 4,
 14,
 0,
 1,
 4,
 14,
 4,
 4,
 1,
 5,
 10,
 14,
 15,
 14,
 4,
 14,
 2,
 14,
 10,
 3,
 14,
 0,
 9,
 8,
 8,
 10,
 14,
 12,
 9,
 4,
 10,
 3,
 3,
 4,
 9,
 9,
 4,
 5,
 2,
 4,
 10,
 10,
 4,
 4,
 4,
 10,
 3,
 3,
 2,
 6,
 9,
 2,
 14,
 5,
 4,
 6,
 3,
 8,
 12,


In [ ]:
dft.iloc[index_fails[:10]]

,uri,date,type,long,lat,atribute,crime,geometry
341,<http://localhost/114499>,2019-03-24,crime:obs,-98.963410,19.314080,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-98.96341 19.31408)
529,<http://localhost/121321>,2018-09-15,crime:obs,-99.220070,19.449890,[<http://localhost/ROBO_A_TRANSEUNTE_EN_VIA_PU...,<http://localhost/ROBO_A_TRANSEUNTE_EN_VÍA_PÚB...,POINT (-99.22007 19.44989)
3080,<http://localhost/243908>,2019-03-24,crime:obs,-98.963410,19.314080,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-98.96341 19.31408)
4693,<http://localhost/335815>,2020-04-12,crime:obs,-99.219958,19.451691,[<http://localhost/ROBO_A_NEGOCIO_SIN_VIOLENCI...,<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (-99.2199581995967 19.4516911973145)
12986,<http://localhost/889358>,2022-08-30,crime:obs,19.323129,19.323129,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (19.3231287787799 19.3231287787799)
12987,<http://localhost/889494>,2022-08-31,crime:obs,19.304446,19.304446,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (19.3044455697613 19.3044455697613)
12988,<http://localhost/889828>,2022-09-01,crime:obs,19.447017,19.447017,[<http://localhost/NARCOMENUDEO_POSESION_SIMPL...,<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (19.4470166110228 19.4470166110228)
12990,<http://localhost/890013>,2022-09-01,crime:obs,19.366314,19.366314,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (19.3663141575609 19.3663141575609)
12991,<http://localhost/890021>,2022-08-28,crime:obs,19.379062,19.379062,[<http://localhost/ROBO_A_TRANSEUNTE_A_BORDO_D...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_T...,POINT (19.3790619853489 19.3790619853489)
12993,<http://localhost/890072>,2022-09-01,crime:obs,19.368912,19.368912,[<http://localhost/ROBO_A_TRANSEUNTE_EN_VIA_PU...,<http://localhost/ROBO_A_TRANSEUNTE_EN_VÍA_PÚB...,POINT (19.3689115621401 19.3689115621401)


In [ ]:
s = (dft["long"] != 0.0) & (dft["lat"] != 0.0)
dft[s][:10]

,uri,date,type,long,lat,atribute,crime,geometry
0,<http://localhost/100091>,2019-06-05,crime:obs,-99.07210,19.35656,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-99.0721 19.35656)
1,<http://localhost/100124>,2019-02-10,crime:obs,-99.14774,19.47544,"[<http://localhost/VIOLENCIA_FAMILIAR>, 38.0, ...",<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (-99.14774 19.47544)
2,<http://localhost/10020>,2019-06-02,crime:obs,-99.10834,19.44424,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.10834 19.44424)
3,<http://localhost/100449>,2019-05-18,crime:obs,-99.17890,19.44453,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.1789 19.44453)
4,<http://localhost/100505>,2019-06-07,crime:obs,-99.13757,19.49107,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.13757 19.49107)
5,<http://localhost/100557>,2019-06-07,crime:obs,-99.19142,19.44123,[<http://localhost/ROBO_A_TRANSEUNTE_A_BORDO_D...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_T...,POINT (-99.19142 19.44123)
6,<http://localhost/100644>,2019-05-28,crime:obs,-99.20802,19.43877,[<http://localhost/ROBO_A_TRANSEUNTE_SALIENDO_...,<http://localhost/ROBO_A_CUENTAHABIENTE_SALIEN...,POINT (-99.20802 19.43877)
7,<http://localhost/100665>,2019-03-08,crime:obs,-99.20259,19.45583,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.20259 19.45583)
9,<http://localhost/100685>,2019-06-08,crime:obs,-99.22376,19.29891,[<http://localhost/HOMICIDIO_POR_ARMA_DE_FUEGO...,<http://localhost/HOMICIDIO_DOLOSO>,POINT (-99.22376 19.29891)
10,<http://localhost/100728>,2019-06-06,crime:obs,-99.07705,19.30740,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.07705 19.3074)


In [ ]:
count = 0
index_fails = []
for n, x in enumerate(lc):
    try:
        gd[x].index[0]
    except:
        # print(x)
        count += 1
        index_fails.append(n)

print("count: ", count)

count:  241


In [ ]:
# index_fails
lc[319].sum()

0

In [ ]:
NAMESPACES = {
        "wtm":  "http://purl.org/heals/food/",
        "ind":  "http://purl.org/heals/ingredient/",
        "recipe":  "https://www.food.com/recipe/",

        "crime": "http://localhost/ontology2#",
        "cube": "http://purl.org/linked-data/cube#",
        # "geo": "http://www.w3.org/2003/01/geo/wgs84_pos#",
        "ontology": "http://dbpedia.org/ontology/",
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "xsd": "http://www.w3.org/2001/XMLSchema#"
    }

kg = kglab.KnowledgeGraph(namespaces = NAMESPACES)
print(f"--> CURRENT WORK DIRECTORY: [{os.getcwd()}]")
_ = kg.load_rdf("/content/tsl/tsl/datasets/raw_files_to_remove/testind3.ttl")


--> CURRENT WORK DIRECTORY: [/content]


In [ ]:
sparql_contiene = """
        SELECT distinct *
        WHERE {
            ?uri a crime:obs .
            ?uri a ?type .
            ?uri crime:tieneFecha ?date .
            ?uri geo1:lat ?lat .
            ?uri geo1:long ?long .

            ?uri crime:contiene ?atribute .
            # ?uri ?x ?y .
        }
    """
# sparql_contiene = "select distinct * where {?uri a crime:obs .}"
df_contiene = kg.query_as_df(sparql=sparql_contiene)
df_contiene

,atribute,uri,date,lat,long,type
0,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_P...,<http://localhost/100091>,2019-06-05,19.35656,-99.07210,crime:obs
1,<http://localhost/VIOLENCIA_FAMILIAR>,<http://localhost/100124>,2019-02-10,19.47544,-99.14774,crime:obs
2,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,<http://localhost/10020>,2019-06-02,19.44424,-99.10834,crime:obs
3,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,<http://localhost/100449>,2019-05-18,19.44453,-99.17890,crime:obs
4,<http://localhost/ROBO_DE_VEHICULO_DE_SERVICIO...,<http://localhost/100505>,2019-06-07,19.49107,-99.13757,crime:obs
...,...,...,...,...,...,...
13995,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_P...,<http://localhost/99795>,2019-10-05,19.39879,-99.05816,crime:obs
13996,<http://localhost/ROBO_A_TRANSEUNTE_EN_VIA_PUB...,<http://localhost/99803>,2019-10-05,19.31211,-99.15714,crime:obs
13997,<http://localhost/VIOLACION>,<http://localhost/99881>,2019-10-06,19.27914,-99.13799,crime:obs
13998,<http://localhost/ROBO_A_TRANSEUNTE_SALIENDO_D...,<http://localhost/9992>,2019-06-03,19.44619,-99.21624,crime:obs


In [ ]:
kg.describe_ns()

,prefix,namespace
0,dct,http://purl.org/dc/terms/
1,prov,http://www.w3.org/ns/prov#
2,schema,https://schema.org/
3,sh,http://www.w3.org/ns/shacl#
4,skos,http://www.w3.org/2004/02/skos/core#
5,wtm,http://purl.org/heals/food/
6,ind,http://purl.org/heals/ingredient/
7,recipe,https://www.food.com/recipe/
8,crime,http://localhost/ontology2#
9,cube,http://purl.org/linked-data/cube#


In [1]:
!pip install geopandas

In [5]:
import pandas as pd
import geopandas as geopd
import json

In [ ]:
# Carga geometrias de alcaldias
f = open("/content/alcaldias_cdmx.json", encoding='utf8')
json_alcaldias = json.load(f)
json_alcaldias

# geo_df = gpd.GeoDataFrame.from_features(json_alcaldias["features"], crs='EPSG:4326')
# geo_df
# # geo_df = geo_df.sort_values("nomgeo")

In [22]:
# Carga geometrias de alcaldias
f = open("/content/agebs.json", encoding='utf8')
json_alcaldias = json.load(f)
json_alcaldias

geo_df = gpd.GeoDataFrame.from_features(json_alcaldias["features"], crs='EPSG:4326')
geo_df["nomgeo"] = geo_df["id"] + "_" + geo_df["cve_ageb"]
del geo_df["id"], geo_df["cve_ageb"]
# geo_df.to_csv("agebs_nomgeo.json", index=False)
geo_df = geo_df.sort_values("nomgeo")
geo_df


,geometry,fid,cve_mun,cve_ent,cvegeo,nomgeo
0,"POLYGON ((2794529.07460 823268.06190, 2794521....",0,014,09,0901400010543,agebs.0_0543
1000,"POLYGON ((2808100.00840 819221.41700, 2808105....",1000,007,09,0900700011636,agebs.1000_1636
1001,"POLYGON ((2809181.30620 822595.06780, 2809182....",1001,007,09,090070001465A,agebs.1001_465A
1002,"POLYGON ((2808663.03720 819221.71530, 2808645....",1002,007,09,0900700011640,agebs.1002_1640
1003,"POLYGON ((2809322.41530 818823.33230, 2809264....",1003,007,09,0900700011655,agebs.1003_1655
...,...,...,...,...,...,...
997,"POLYGON ((2806677.60270 820827.59140, 2806728....",997,007,09,0900700011246,agebs.997_1246
998,"POLYGON ((2802688.65490 819966.17130, 2802753....",998,007,09,0900700011369,agebs.998_1369
999,"POLYGON ((2812616.26120 819840.50240, 2812662....",999,007,09,0900700012691,agebs.999_2691
99,"POLYGON ((2799975.70230 822567.41970, 2799968....",99,014,09,0901400010666,agebs.99_0666
